In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
#解决中文显示问题
plt.rcParams['font.sans-serif'] = ['KaiTi'] # 指定默认字体
plt.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
#图像显示不清楚
%config InlineBackend.figure_format ='svg'

## 1.处理PM2.5

In [2]:
read_path="E:/33-数学建模资料/研究生数学建模竞赛-2021/数据.xlsx"
to_path="E:/33-数学建模资料/研究生数学建模竞赛-2021/process_nan_anomaly.xlsx"
data1=pd.read_excel(read_path,sheet_name='PM2.5')
writer=pd.ExcelWriter(to_path)
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   时间            270 non-null    datetime64[ns]
 1   地点            270 non-null    object        
 2   粒径            270 non-null    object        
 3   颗粒物浓度（ug/m3）  269 non-null    float64       
 4   OC(ug/m3)     269 non-null    float64       
 5   EC(ug/m3)     269 non-null    float64       
 6   OC/EC         269 non-null    float64       
 7   OC1(ug/m3)    269 non-null    float64       
 8   OC2(ug/m3)    269 non-null    float64       
 9   OC3(ug/m3)    269 non-null    float64       
 10  OC4(ug/m3)    269 non-null    float64       
 11  EC1(ug/m3)    269 non-null    float64       
 12  EC2(ug/m3)    269 non-null    float64       
 13  EC3(ug/m3)    269 non-null    float64       
 14  OPC(ug/m3)    104 non-null    float64       
dtypes: datetime64[ns](1), float64(12), objec

In [3]:
#OPC列缺失值较多，用均值填充
mean_val=data1['OPC(ug/m3)'].mean()
data1['OPC(ug/m3)'].fillna(mean_val,inplace=True)

In [4]:
data1=data1.dropna(axis=0,how='any')#缺失值较少，删除

In [5]:
#去除有负值和0值的行
# data1.drop(index = data1.EC3(ug/m3)[df1.EC3(ug/m3) < 0].index )
def detection(data):
    columns_=['颗粒物浓度（ug/m3）','OC(ug/m3)','EC(ug/m3)','OC/EC','OC1(ug/m3)','OC2(ug/m3)','OC3(ug/m3)','OC4(ug/m3)','EC1(ug/m3)','EC2(ug/m3)','EC3(ug/m3)','OPC(ug/m3)']
    index=[]
    for i in columns_:
        index.append(data[data[i]<= 0].index)
    print(index)

In [6]:
detection(data1)

[Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([216, 217, 219], dtype='int64'), Int64Index([16], dtype='int64'), Int64Index([], dtype='int64')]


In [7]:
data1=data1.drop([16,216, 217, 219])

In [8]:
#异常值处理：这里的异常值有负值

In [9]:
def AddQuater(data):
    data.insert(1,'季度',data['时间'].dt.quarter)
    PM25_count={}
    for i in data['季度']:
        PM25_count[i]=PM25_count.get(i,0)+1
    PM25_count=sorted(PM25_count.items())
    PM25_count=dict(PM25_count)
    return PM25_count#返回季度和对应天数

In [10]:
def to_file(data,str1):
    data.to_excel(writer,sheet_name=str1)

In [11]:
PM25_count=AddQuater(data1)   
to_file(data1,'PM2.5')

## 2.处理PM1

In [12]:
data2=pd.read_excel(read_path,sheet_name='PM1')
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   时间            206 non-null    datetime64[ns]
 1   地点            206 non-null    object        
 2   粒径            206 non-null    object        
 3   颗粒物浓度（ug/m3）  190 non-null    float64       
 4   OC(ug/m3)     206 non-null    float64       
 5   EC(ug/m3)     206 non-null    float64       
 6   OC/EC         206 non-null    float64       
 7   OC1(ug/m3)    206 non-null    float64       
 8   OC2(ug/m3)    206 non-null    float64       
 9   OC3(ug/m3)    206 non-null    float64       
 10  OC4(ug/m3)    206 non-null    float64       
 11  EC1(ug/m3)    206 non-null    float64       
 12  EC2(ug/m3)    206 non-null    float64       
 13  EC3(ug/m3)    205 non-null    float64       
 14  OPC(ug/m3)    206 non-null    float64       
dtypes: datetime64[ns](1), float64(12), objec

In [13]:
#处理缺失值和异常值
data2=data2.dropna(axis=0,how='any')#删除缺失值
detection(data2)#检测异常值

[Int64Index([87], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([149], dtype='int64'), Int64Index([149], dtype='int64')]


In [14]:
data2=data2.drop([87,149])

In [15]:
PM1_count=AddQuater(data2)   
to_file(data2,'PM1')

## 3.处理PM10

In [16]:
data3=pd.read_excel(read_path,sheet_name='PM10')
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   时间            194 non-null    datetime64[ns]
 1   地点            194 non-null    object        
 2   粒径            194 non-null    object        
 3   颗粒物浓度（ug/m3）  192 non-null    float64       
 4   OC(ug/m3)     194 non-null    float64       
 5   EC(ug/m3)     194 non-null    float64       
 6   OC/EC         194 non-null    float64       
 7   OC1(ug/m3)    194 non-null    float64       
 8   OC2(ug/m3)    194 non-null    float64       
 9   OC3(ug/m3)    194 non-null    float64       
 10  OC4(ug/m3)    194 non-null    float64       
 11  EC1(ug/m3)    194 non-null    float64       
 12  EC2(ug/m3)    194 non-null    float64       
 13  EC3(ug/m3)    194 non-null    float64       
 14  OPC(ug/m3)    194 non-null    float64       
dtypes: datetime64[ns](1), float64(12), objec

In [17]:
#处理缺失值和异常值
data3=data3.dropna(axis=0,how='any')
detection(data3)

[Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([185], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([], dtype='int64'), Int64Index([26], dtype='int64'), Int64Index([], dtype='int64')]


In [18]:
data3=data3.drop([185,26])

In [19]:
PM10_count=AddQuater(data3)   
to_file(data3,'PM10')

In [20]:
writer.save()
writer.close()